<a href="https://colab.research.google.com/github/allanzhou777/Adversarial-Attacks/blob/main/Transfer_Learning_on_Robust_Imagenet_on_Breast_Ultrasound_Resnet18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install cox
!pip install tensorboardX
!pip install apex
!git clone https://github.com/MadryLab/robustness.git
!pip install dill
!pip install pyngrok

import os
os.chdir("/content/robustness/")
import torch
import torch as ch
from torch import nn

import dill

from cox.utils import Parameters
from cox.readers import CollectionReader
import cox.store
import copy
from robustness import imagenet_models # or cifar_models
from robustness import model_utils, train, defaults

from robustness.attacker import AttackerModel
from robustness.datasets import ImageNet
from robustness.datasets import DATASETS
# from robustness.datasets import MyNewDataSet
from robustness.model_utils import make_and_restore_model
from robustness.defaults import check_and_fill_args
from robustness.tools import constants, helpers
from robustness.tools.custom_modules import SequentialWithArgs
from torchvision.models import resnet18
from robustness.model_utils import make_and_restore_model
from argparse import ArgumentParser
import datetime
from fractions import Fraction
import plotly.graph_objects as go
import pandas as pd

CURRENT_DIRECTORY = os.getcwd()
print(f'current directory{CURRENT_DIRECTORY}')



Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.8/709.8 kB 29.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━

/content/robustness/robustness/train.py:24: UserWarning: Could not import amp.
  warnings.warn('Could not import amp.')


In [ ]:
# CONSTANTS
IMAGES_PATH = '/content/dataset/Dataset_BUSI_with_GT/'

# unzip Breast Ultrasound dataset
!cp /content/drive/MyDrive/Adversarial\ Attacks/datasets/breast\ ultrasound/breasts_ultrasound.zip /content/
!unzip /content/breasts_ultrasound.zip -d /content/dataset/

Archive:  /content/breasts_ultrasound.zip
  inflating: /content/dataset/Dataset_BUSI_with_GT/benign/benign (1).png  
  inflating: /content/dataset/Dataset_BUSI_with_GT/benign/benign (1)_mask.png  
  inflating: /content/dataset/Dataset_BUSI_with_GT/benign/benign (10).png  
  inflating: /content/dataset/Dataset_BUSI_with_GT/benign/benign (10)_mask.png  
  inflating: /content/dataset/Dataset_BUSI_with_GT/benign/benign (100).png  
  inflating: /content/dataset/Dataset_BUSI_with_GT/benign/benign (100)_mask.png  
  inflating: /content/dataset/Dataset_BUSI_with_GT/benign/benign (100)_mask_1.png  
  inflating: /content/dataset/Dataset_BUSI_with_GT/benign/benign (101).png  
  inflating: /content/dataset/Dataset_BUSI_with_GT/benign/benign (101)_mask.png  
  inflating: /content/dataset/Dataset_BUSI_with_GT/benign/benign (102).png  
  inflating: /content/dataset/Dataset_BUSI_with_GT/benign/benign (102)_mask.png  
  inflating: /content/dataset/Dataset_BUSI_with_GT/benign/benign (103).png  
  inflat

## Helper Functions


In [ ]:
#first bracket: eps-train; second bracket: eps-test
L2_EVAL = 'l2_eval_{}_{}/'
L_INF_EVAL = 'l_inf_eval_{}_{}/'
UNCONSTRAINED_EVAL = 'unconstrained_eval/'
FOURIER_EVAL = 'fourier_eval/'
RANDOM_EVAL = 'random_eval/'


#HELPER FUNCTIONS
calc_attack_lr = lambda eps_test, steps: 2.5 * eps_test / steps


def setup_store_with_metadata(args):
    '''
    Sets up a store for training according to the arguments object. See the
    argparse object above for options.
    '''
    # Create the store
    store = cox.store.Store(args.out_dir, args.exp_name)
    args_dict = args.__dict__
    schema = cox.store.schema_from_dict(args_dict)
    store.add_table('metadata', schema)
    store['metadata'].append_row(args_dict)

    return store


def evaluate_robust_model(eval_args,
                          model,
                          test_out_dir,
                          attack_steps=[100]):
  '''
  Evaluates a robust neural network on adversarial attacks. Stores results in Store in test_out_dir.
  Args:
    eval_args: (cox.utils.Parameters) hyperparameters used to evaluate the robust model
        https://github.com/MadryLab/robustness/blob/master/robustness/defaults.py#L91
    model: (robustness.attacker.AttackerModel) robust neural network
    test_out_dir: (str) where evaluation results are stored
    attack_steps: (optional, list) stores options for number of attack steps
  Returns: None
  '''
  assert eval_args.__contains__('eps'), "eps value required in eval_args parameter"
  assert eval_args.__contains__('constraint'), "constraint value required in eval_args parameter"

  for atk_steps in attack_steps:
    print('evaluating robust model with constaint: {}, eps: {}, and attack steps: {}'.format(eval_args.constraint,
                                                              Fraction(eval_args.eps).limit_denominator(),
                                                              atk_steps))

    eval_args.__setattr__('attack_steps', atk_steps)
    eval_args.__setattr__('attack_lr', calc_attack_lr(eval_args.__getattr__('eps'),
                                                    eval_args.__getattr__('attack_steps')))
    eval_args.__setattr__('out_dir', test_out_dir)

    # check whether the evaluation args provided are correct
    train.check_required_args(eval_args, eval_only=True)
    test_loader = copy.copy(val_loader)
    print(f'test results dir: {test_out_dir}')

    test_store = setup_store_with_metadata(eval_args)

    train.eval_model(eval_args, model, test_loader, test_store)
    test_store.close()

In [ ]:
def load_split_train_test(args, datadir, valid_size = .2):
    train_data = datasets.ImageFolder(datadir,
                    transform=data_transforms['train']) #Picks up Image Paths from its respective folders and label them
    test_data = datasets.ImageFolder(datadir,
                    transform=data_transforms['val'])
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    train_idx, test_idx = indices[split:], indices[:split]
    dataset_size = {"train":len(train_idx), "val":len(test_idx)}
    train_sampler = SubsetRandomSampler(train_idx) # Sampler for splitting train and val images
    test_sampler = SubsetRandomSampler(test_idx)
    trainloader = ch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=args.batch_size) # DataLoader provides data from traininng and validation in batches
    testloader = ch.utils.data.DataLoader(test_data,
                   sampler=test_sampler, batch_size=args.batch_size)
    return trainloader, testloader, dataset_size

In [ ]:
def ft(model_name, model_ft, num_classes, additional_hidden=0):
    if model_name in ["resnet", "resnet18", "resnet50", "wide_resnet50_2", "wide_resnet50_4", "resnext50_32x4d", 'shufflenet']:
        num_ftrs = model_ft.fc.in_features
        # The two cases are split just to allow loading
        # models trained prior to adding the additional_hidden argument
        # without errors
        if additional_hidden == 0:
            model_ft.fc = nn.Linear(num_ftrs, num_classes)
        else:
            model_ft.fc = SequentialWithArgs(
                *list(sum([[nn.Linear(num_ftrs, num_ftrs), nn.ReLU()] for i in range(additional_hidden)], [])),
                nn.Linear(num_ftrs, num_classes)
            )
        input_size = 224
    elif model_name == "alexnet":
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224
    elif "vgg" in model_name:
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224
    elif model_name == "squeezenet":
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224
    elif model_name == "densenet":
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224
    elif model_name in ["mnasnet", "mobilenet"]:
        num_ftrs = model_ft.classifier[1].in_features
        model_ft.classifier[1] = nn.Linear(num_ftrs, num_classes)
        input_size = 224
    else:
        raise ValueError("Invalid model type, exiting...")

    return model_ft

In [ ]:
eval_args = Parameters({
    'adv_eval': 1,
    'eval_only': 1,
    'use_best': True,
    'random_restarts': 0,
})

In [ ]:
BLUE_SHADE = 'rgba(0, 0, 255, .2)'
RED_SHADE = 'rgba(255, 0, 0, .2)'
YELLOW_SHADE = 'rgba(255,255,0,0.2)'
PURPLE_SHADE = 'rgba(216,191,216,0.2)'
GREEN_SHADE = 'rgba(0,128,0, .2)'
ORANGE_SHADE = 'rgba(255,128,0, .2)'


colors = ['red', 'blue', 'green', 'orange']
def measurement_scatter(mean: pd.Series,
                        x: pd.Series,
                        upper: pd.Series,
                        lower: pd.Series,
                        col: str,
                        title: str,
                        color: str):


  shade = BLUE_SHADE
  if color == 'red':
    shade = RED_SHADE
  elif color == 'yellow':
    shade = YELLOW_SHADE
  elif color == 'purple':
    shade = PURPLE_SHADE
  elif color == 'green':
    shade = GREEN_SHADE
  elif color == 'orange':
    shade = ORANGE_SHADE
  return [go.Scatter(
        x=x,
        y=mean,
        line=dict(color=color),
        mode='lines+markers',
        name=title,
    )]

In [ ]:
#parser
parser = ArgumentParser()
parser.add_argument('-f')
parser = defaults.add_args_to_parser(defaults.MODEL_LOADER_ARGS, parser)
parser = defaults.add_args_to_parser(defaults.TRAINING_ARGS, parser)
parser = defaults.add_args_to_parser(defaults.PGD_ARGS, parser)
# Note that we can add whatever extra arguments we want to the parser here
args = parser.parse_args()

In [ ]:
#Statistics Based on ImageNet Data for Normalisation
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
mean_nums = [0.485, 0.456, 0.406]
std_nums = [0.229, 0.224, 0.225]

data_transforms = {"train":transforms.Compose([
                                transforms.Resize((150,150)), #Resizes all images into same dimension
                                # transforms.RandomRotation(10), # Rotates the images upto Max of 10 Degrees
                                # transforms.RandomHorizontalFlip(p=0.4), #Performs Horizantal Flip over images
                                transforms.ToTensor(), # Coverts into Tensors
                                #transforms.Normalize(mean = mean_nums, std=std_nums)
                    ]), # Normalizes
                    "val": transforms.Compose([
                                transforms.Resize((150,150)),
                                transforms.CenterCrop(150), #Performs Crop at Center and resizes it to 150x150
                                transforms.ToTensor(),
                                # transforms.Normalize(mean=mean_nums, std = std_nums)
                    ])}

In [ ]:
ARCH = 'resnet_18'
OUT_DIR = '/content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/' + ARCH

# Train Model

In [ ]:
#define default training parameters
train_kwargs = {
    'adv_train': 0,
    'dataset': 'imagenet',
    'out_dir': OUT_DIR,
    'arch': 'resnet18',
    'epochs': 20,
    'mixed_precision': 0,
    'batch_size': 32,
    'log_iters': 1,
    'lr': 1e-3,
    'additional_hidden': 1,
}
args = Parameters(train_kwargs)
#sanity checks
ds_class = DATASETS[args.dataset]
assert args.dataset is not None, "Must provide a dataset"
args = check_and_fill_args(args, defaults.TRAINING_ARGS, ds_class)

if args.adv_train or args.adv_eval:
  args = check_and_fill_args(args, defaults.PGD_ARGS, ds_class)
args = check_and_fill_args(args, defaults.MODEL_LOADER_ARGS, ds_class)
print(f'Training Arguments: ', args)
print(train_kwargs)

Training Arguments:  {
  "adv_train": 0,
  "dataset": "imagenet",
  "out_dir": "/content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18",
  "arch": "resnet18",
  "epochs": 20,
  "mixed_precision": 0,
  "batch_size": 32,
  "log_iters": 1,
  "lr": 0.001,
  "additional_hidden": 1,
  "weight_decay": 0.0001,
  "momentum": 0.9,
  "step_lr": 50,
  "step_lr_gamma": 0.1,
  "lr_interpolation": "step",
  "save_ckpt_iters": -1,
  "data": "/tmp/",
  "workers": 30,
  "resume_optimizer": 0,
  "data_aug": 1
}
{'adv_train': 0, 'dataset': 'imagenet', 'out_dir': '/content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18', 'arch': 'resnet18', 'epochs': 20, 'mixed_precision': 0, 'batch_size': 32, 'log_iters': 1, 'lr': 0.001, 'additional_hidden': 1, 'weight_decay': 0.0001, 'momentum': 0.9, 'step_lr': 50, 'step_lr_gamma': 0.1, 'lr_interpolation': 'step', 'save_ckpt_iters': -1, 'data': '/tmp/', 'workers': 30, 'resume_optimizer': 0, 'data_aug': 1}


In [ ]:
train_data = datasets.ImageFolder(IMAGES_PATH,
                    transform=data_transforms['train']) #Picks up Image Paths from its respective folders and label them
test_data = datasets.ImageFolder(IMAGES_PATH,
                    transform=data_transforms['val'])

train_loader, val_loader, dataset_size = load_split_train_test(args, IMAGES_PATH, .2)
dataloaders = {"train":train_loader, "val":val_loader}
data_sizes = {x: len(dataloaders[x].sampler) for x in ['train','val']}
class_names = train_loader.dataset.classes
print(class_names)

['benign', 'malignant', 'normal']


# Define Model

##Training the model

In [ ]:
dataset = ImageNet(data_path = "")
# dataset = MyNewDataSet('/path/to/dataset/') #custom dataset path here
imagenet = dataset

model_path = "resnet18_l2_eps0.ckpt"
# model_path = "resnet18_l2_eps0_5.ckpt"
# model_path = "resnet18_l2_eps1.ckpt"
# model_path = "resnet18_l2_eps3.ckpt"
# model_path = "resnet18_l2_eps5.ckpt"

full_model_path = "/content/drive/MyDrive/Adversarial Attacks/Robust ImageNet Transfer Models/" + model_path
model, _ = make_and_restore_model(arch = resnet18(False), dataset = dataset, resume_path = full_model_path, add_custom_forward=False, pytorch_pretrained=False)
checkpoint = None

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


=> loading checkpoint '/content/drive/MyDrive/Adversarial Attacks/Robust ImageNet Transfer Models/resnet18_l2_eps0.ckpt'
=> loaded checkpoint '/content/drive/MyDrive/Adversarial Attacks/Robust ImageNet Transfer Models/resnet18_l2_eps0.ckpt' (epoch 90)


In [ ]:
# if not args.no_replace_last_layer and not args.eval_only:
print(f'[Replacing the last layer with {args.additional_hidden} '
      f'hidden layers and 1 classification layer that fits the {args.dataset} dataset.]')
while hasattr(model, 'model'):
    model = model.model
model = ft("resnet", model, 3, False)
model, checkpoint = model_utils.make_and_restore_model(arch=model, dataset=imagenet,
                                                        add_custom_forward=args.additional_hidden > 0 or False)

[Replacing the last layer with 1 hidden layers and 1 classification layer that fits the imagenet dataset.]


In [ ]:
try:
  st = datetime.datetime.now()

  out_store = cox.store.Store(OUT_DIR)

  train.train_model(args, model, (train_loader, val_loader), store=out_store)

  end = datetime.datetime.now()
  total = end - st
  print(total)
  out_store.close()
except Exception as e:
  out_store.close()
  print(f"model did not train to completion. \n error: {e}")

Logging in: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18/061aba6a-971b-4d7c-816d-ae40c78d08f1


Train Epoch:0 | Loss 0.5729 | NatPrec1 75.851 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 40/40 [00:21<00:00,  1.84it/s]
Val Epoch:0 | Loss 0.4294 | NatPrec1 82.222 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 10/10 [00:02<00:00,  4.30it/s]
Train Epoch:1 | Loss 0.2788 | NatPrec1 90.103 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 40/40 [00:12<00:00,  3.23it/s]
Val Epoch:1 | Loss 0.2791 | NatPrec1 88.571 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 10/10 [00:02<00:00,  4.36it/s]
Train Epoch:2 | Loss 0.1428 | NatPrec1 95.249 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 40/40 [00:12<00:00,  3.20it/s]
Val Epoch:2 | Loss 0.2872 | NatPrec1 87.619 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 10/10 [00:02<00:00,  4.35it/s]
Train Epoch:3 | Loss 0.1022 | NatPrec1 97.150 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 40/40 [00:12<00:00,  3.26it/s]
Val Epoch:3 | Loss 0.2467 | NatPrec1 90.794 | NatPrec5 100.000 | Reg term: 

0:05:26.555680


# Experiment Information

EXP IDs


ResNet18 L2 eps= 0.0
8f2dbe44-cfef-4b2e-a4ef-b3badf9e6c8b
061aba6a-971b-4d7c-816d-ae40c78d08f1

ResNet18 L2 eps= 0.5
ffb4bcb6-c003-4c28-bc32-42b14fafc2cc

ResNet18 L2 eps= 1.0
35f5a464-9f3d-4eb6-82ab-876bb44d22c2

ResNet18 L2 eps= 3.0
f2cbfe1d-af9e-4bc8-80dd-b9055816c973

ResNet18 L2 eps= 5.0
181e280b-f700-4835-b3e9-5e52c8cf5b6c





In [ ]:
EXP = "061aba6a-971b-4d7c-816d-ae40c78d08f1"
BEST = "checkpoint.pt.best"
LATEST = "checkpoint.pt.latest"
FIRST = "0_checkpoint.pt"

In [ ]:
reader = CollectionReader(OUT_DIR)
logs = reader.df('logs')
reader.close()
logs = logs[logs.exp_id == EXP]
logs.head()

100%|██████████| 33/33 [00:11<00:00,  2.80it/s]

,epoch,nat_prec1,adv_prec1,nat_loss,adv_loss,train_prec1,train_loss,time,exp_id
0,1,82.222221,-1.0,0.429409,-1.0,75.851151,0.572918,24.137678,061aba6a-971b-4d7c-816d-ae40c78d08f1
1,2,88.571426,-1.0,0.279137,-1.0,90.102928,0.278764,39.466406,061aba6a-971b-4d7c-816d-ae40c78d08f1
2,3,87.619049,-1.0,0.287175,-1.0,95.249405,0.142836,55.172428,061aba6a-971b-4d7c-816d-ae40c78d08f1
3,4,90.793655,-1.0,0.246693,-1.0,97.149643,0.102161,70.084469,061aba6a-971b-4d7c-816d-ae40c78d08f1
4,5,91.428574,-1.0,0.222707,-1.0,98.574821,0.057773,85.602184,061aba6a-971b-4d7c-816d-ae40c78d08f1


In [ ]:
# validation accuracies and loss
nat_prec1 = logs.nat_prec1
adv_prec1 = logs.adv_prec1
nat_loss = logs.nat_loss
adv_loss = logs.adv_loss

# training accuracies and losses --> here, we only have adversarial accuracies and losses...do you understand why?
train_prec1 = logs.train_prec1
train_loss = logs.train_loss

In [ ]:
losses = ['train_loss', 'nat_loss', 'adv_loss']

scatter = []
for i, loss in enumerate(losses):
  scatter = scatter + measurement_scatter(logs[loss], logs.epoch, logs[loss], logs[loss], loss, loss, color=colors[i])


fig = go.Figure(scatter)

fig.update_layout(
    yaxis_title='CE Loss',
    xaxis_title='Epoch',
    hovermode="x",
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    font={'size': 20}
)

fig.show()

In [ ]:
accuracies = ['train_prec1', 'nat_prec1', 'adv_prec1']

scatter = []
for i, acc in enumerate(accuracies):
  print(f'Accuracy: {loss}...')
  scatter = scatter + measurement_scatter(logs[acc], logs.epoch, logs[acc], logs[acc], acc, acc, color=colors[i])


fig = go.Figure(scatter)

fig.update_layout(
    yaxis_title='Accuracy',
    xaxis_title='Epoch',
    hovermode="x",
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    font={'size': 20}
)

fig.show()

Accuracy: adv_loss...
Accuracy: adv_loss...
Accuracy: adv_loss...


#Evaluate Model

In [ ]:
resume_path=OUT_DIR + "/" + EXP + "/" + BEST

In [ ]:
from robustness import imagenet_models # or cifar_models
from robustness.datasets import ImageNet

model = AttackerModel(resnet18(), imagenet)

while hasattr(model, 'model'):
    model = model.model
model = ft("resnet", model, 3, False)

model, checkpoint = model_utils.make_and_restore_model(arch=model, dataset=imagenet,
                                                        add_custom_forward=args.additional_hidden > 0 or False,
                                                       resume_path = resume_path)

=> loading checkpoint '/content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18/061aba6a-971b-4d7c-816d-ae40c78d08f1/checkpoint.pt.best'
=> loaded checkpoint '/content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18/061aba6a-971b-4d7c-816d-ae40c78d08f1/checkpoint.pt.best' (epoch 12)


##Evaluating robust model with constaint: 2, ϵ-test: 0, 0.25, 0.5, 1.0, 2.0, and attack_steps: [20, 200]


####ϵ-test: 0


In [ ]:
from pyngrok import ngrok

# Set the ngrok authtoken (only needed for free users)
NGROK_AUTH_TOKEN = "23Qif7HL0weneUkOUXkCzY3ZRQG_3gPVLhxnC1kRDVRqWezY7"

ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open a tunnel to the TensorBoard port
public_url = ngrok.connect(addr="6006", proto="http")

public_url

<NgrokTunnel: "https://5abb-35-203-136-58.ngrok-free.app" -> "http://localhost:6006">

In [ ]:
eval_args.__setattr__('constraint', '2')
eval_args.__setattr__('eps', 0.0)
print(f'evaluation args: {eval_args}')

#evals on robust model
TEST_OUT = OUT_DIR + L2_EVAL.format(.5, 0) + 'robustness'
evaluate_robust_model(eval_args, model, TEST_OUT)

evaluation args: {
  "adv_eval": 1,
  "eval_only": 1,
  "use_best": true,
  "random_restarts": 0,
  "constraint": "2",
  "eps": 0.0
}
evaluating robust model with constaint: 2, eps: 0, and attack steps: 100
test results dir: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_0/robustness
Logging in: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_0/robustness/e3447f40-4774-4b1c-8753-368de999d476


Val Epoch:0 | Loss 0.2591 | NatPrec1 92.063 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 10/10 [00:03<00:00,  2.51it/s]
Val Epoch:0 | Loss 0.2591 | AdvPrec1 92.063 | AdvPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 10/10 [00:39<00:00,  3.98s/it]


In [ ]:
!tensorboard --logdir=/content/drive/MyDrive/Adversarial\ Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_0/robustness/e3447f40-4774-4b1c-8753-368de999d476 serve

2023-07-10 15:08:00.367615: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-10 15:08:01.169756: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-10 15:08:02.750382: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-10 15:08:02.783219: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

####ϵ-test: 0.25


In [ ]:
eval_args.__setattr__('constraint', '2')
eval_args.__setattr__('eps', 0.25)
print(f'evaluation args: {eval_args}')

#evals on robust model
TEST_OUT = OUT_DIR + L2_EVAL.format(.5, 0.25) + 'robustness'
evaluate_robust_model(eval_args, model, TEST_OUT)

evaluation args: {
  "adv_eval": 1,
  "eval_only": 1,
  "use_best": true,
  "random_restarts": 0,
  "constraint": "2",
  "eps": 0.25,
  "attack_steps": 100,
  "attack_lr": 0.0,
  "out_dir": "/content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_0/robustness"
}
evaluating robust model with constaint: 2, eps: 1/4, and attack steps: 100
test results dir: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_0.25/robustness
Logging in: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_0.25/robustness/0dd70b9e-7744-42b8-8ae8-772ef07577ba


Val Epoch:0 | Loss 0.2591 | NatPrec1 92.063 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 10/10 [00:03<00:00,  2.91it/s]
Val Epoch:0 | Loss 11.4292 | AdvPrec1 14.921 | AdvPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 10/10 [00:41<00:00,  4.15s/it]


In [ ]:
!tensorboard --logdir=/content/drive/MyDrive/Adversarial\ Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_0.25/robustness/0dd70b9e-7744-42b8-8ae8-772ef07577ba serve

2023-07-10 15:05:46.855383: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-10 15:05:48.014300: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-10 15:05:50.023299: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-10 15:05:50.055791: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

####ϵ-test: 0.5


In [ ]:
eval_args.__setattr__('constraint', '2')
eval_args.__setattr__('eps', 0.5)
print(f'evaluation args: {eval_args}')

#evals on robust model
TEST_OUT = OUT_DIR + L2_EVAL.format(.5, 0.5) + 'take_one'
evaluate_robust_model(eval_args, model, TEST_OUT)

evaluation args: {
  "adv_eval": 1,
  "eval_only": 1,
  "use_best": true,
  "random_restarts": 0,
  "constraint": "2",
  "eps": 0.5,
  "attack_steps": 100,
  "attack_lr": 0.00625,
  "out_dir": "/content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_0.25/robustness"
}
evaluating robust model with constaint: 2, eps: 1/2, and attack steps: 100
test results dir: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_0.5/take_one
Logging in: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_0.5/take_one/e052934b-61ee-4c7f-81b9-90aebb124876


Val Epoch:0 | Loss 0.2591 | NatPrec1 92.063 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 10/10 [00:03<00:00,  3.11it/s]
Val Epoch:0 | Loss 25.1929 | AdvPrec1 0.317 | AdvPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 10/10 [00:38<00:00,  3.88s/it]


In [ ]:
!tensorboard --logdir=/content/drive/MyDrive/Adversarial\ Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_0.5/take_one/e052934b-61ee-4c7f-81b9-90aebb124876 serve

2023-07-10 15:14:09.035696: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-10 15:14:10.186911: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-10 15:14:12.422267: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-10 15:14:12.466246: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

####ϵ-test: 1.0


In [ ]:
eval_args.__setattr__('constraint', '2')
eval_args.__setattr__('eps', 1.0)
print(f'evaluation args: {eval_args}')

#evals on robust model
TEST_OUT = OUT_DIR + L2_EVAL.format(.5, 1.0) + 'test_1.0'
evaluate_robust_model(eval_args, model, TEST_OUT)

evaluation args: {
  "adv_eval": 1,
  "eval_only": 1,
  "use_best": true,
  "random_restarts": 0,
  "constraint": "2",
  "eps": 1.0,
  "attack_steps": 100,
  "attack_lr": 0.0125,
  "out_dir": "/content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_0.5/take_one"
}
evaluating robust model with constaint: 2, eps: 1, and attack steps: 100
test results dir: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_1.0/test_1.0
Logging in: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_1.0/test_1.0/edf9d619-a6ff-48a6-86bf-db6e14b03026


Val Epoch:0 | Loss 0.2591 | NatPrec1 92.063 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 10/10 [00:02<00:00,  4.09it/s]
Val Epoch:0 | Loss 40.2829 | AdvPrec1 0.000 | AdvPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 10/10 [00:37<00:00,  3.80s/it]


In [ ]:
!tensorboard --logdir=/content/drive/MyDrive/Adversarial\ Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_1.0/test_1.0/edf9d619-a6ff-48a6-86bf-db6e14b03026 serve

2023-07-10 15:16:35.198996: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-10 15:16:36.420715: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2023-07-10 15:16:38.724556: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-10 15:16:38.763947: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-10 15:16:38.764317: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

####ϵ-test: 2.0


In [ ]:
eval_args.__setattr__('constraint', '2')
eval_args.__setattr__('eps', 2.0)
print(f'evaluation args: {eval_args}')

#evals on robust model
TEST_OUT = OUT_DIR + L2_EVAL.format(.5, 2.0) + 'eps_2.0'
evaluate_robust_model(eval_args, model, TEST_OUT)

evaluation args: {
  "adv_eval": 1,
  "eval_only": 1,
  "use_best": true,
  "random_restarts": 0,
  "constraint": "2",
  "eps": 2.0,
  "attack_steps": 100,
  "attack_lr": 0.025,
  "out_dir": "/content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_1.0/test_1.0"
}
evaluating robust model with constaint: 2, eps: 2, and attack steps: 100
test results dir: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_2.0/eps_2.0
Logging in: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_2.0/eps_2.0/8b89c7eb-e0ae-4338-827d-1b7154616491


Val Epoch:0 | Loss 0.2591 | NatPrec1 92.063 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 10/10 [00:03<00:00,  3.20it/s]
Val Epoch:0 | Loss 52.0509 | AdvPrec1 0.000 | AdvPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 10/10 [00:38<00:00,  3.89s/it]


In [ ]:
!tensorboard --logdir=/content/drive/MyDrive/Adversarial\ Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_2.0/eps_2.0/8b89c7eb-e0ae-4338-827d-1b7154616491 serve

2023-07-10 15:19:37.463703: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-10 15:19:38.264012: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-10 15:19:39.815572: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-10 15:19:39.846415: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

##Evaluating robust model with constaint: ∞, ϵ-test: 0, 8/255, 16/255, and attack_steps: [20, 200]


In [ ]:
from pyngrok import ngrok

# Set the ngrok authtoken (only needed for free users)
NGROK_AUTH_TOKEN = "23Qif7HL0weneUkOUXkCzY3ZRQG_3gPVLhxnC1kRDVRqWezY7"

ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open a tunnel to the TensorBoard port
public_url = ngrok.connect(addr="6006", proto="http")

public_url

<NgrokTunnel: "https://d877-35-203-136-58.ngrok-free.app" -> "http://localhost:6006">

####ϵ-test: 0


In [ ]:
eval_args.__setattr__('constraint', 'inf')
eval_args.__setattr__('eps', 0)
print(f'evaluation args: {eval_args}')

#evals on robust model
TEST_L_INF_OUT = OUT_DIR + L_INF_EVAL.format(8/255, 0) + 'linf'
evaluate_robust_model(eval_args, model, TEST_L_INF_OUT)

evaluation args: {
  "adv_eval": 1,
  "eval_only": 1,
  "use_best": true,
  "random_restarts": 0,
  "constraint": "inf",
  "eps": 0,
  "attack_steps": 100,
  "attack_lr": 0.05,
  "out_dir": "/content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l2_eval_0.5_2.0/eps_2.0"
}
evaluating robust model with constaint: inf, eps: 0, and attack steps: 100
test results dir: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l_inf_eval_0.03137254901960784_0/linf
Logging in: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l_inf_eval_0.03137254901960784_0/linf/caf9b369-7bb1-4a59-9374-7d0d27c1f8fe


Val Epoch:0 | Loss 0.2591 | NatPrec1 92.063 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 10/10 [00:02<00:00,  4.16it/s]
Val Epoch:0 | Loss 0.2591 | AdvPrec1 92.063 | AdvPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 10/10 [00:37<00:00,  3.78s/it]


In [ ]:
!tensorboard --logdir=/content/drive/MyDrive/Adversarial\ Attacks/robust_transfer_learning/resnet_18l_inf_eval_0.03137254901960784_0/linf/caf9b369-7bb1-4a59-9374-7d0d27c1f8fe serve

2023-07-10 15:22:07.812547: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2023-07-10 15:22:08.980343: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2023-07-10 15:22:10.795088: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-10 15:22:10.824371: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-10 15:22:10.824627: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

####ϵ-test: 8/255


In [ ]:
eval_args.__setattr__('constraint', 'inf')
eval_args.__setattr__('eps', 8/255)
print(f'evaluation args: {eval_args}')

#evals on robust model
TEST_L_INF_OUT = OUT_DIR + L_INF_EVAL.format(8/255, 8/255) + 'linf'
evaluate_robust_model(eval_args, model, TEST_L_INF_OUT)

evaluation args: {
  "adv_eval": 1,
  "eval_only": 1,
  "use_best": true,
  "random_restarts": 0,
  "constraint": "inf",
  "eps": 0.03137254901960784,
  "attack_steps": 100,
  "attack_lr": 0.0,
  "out_dir": "/content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l_inf_eval_0.03137254901960784_0/linf"
}
evaluating robust model with constaint: inf, eps: 8/255, and attack steps: 100
test results dir: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l_inf_eval_0.03137254901960784_0.03137254901960784/linf
Logging in: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l_inf_eval_0.03137254901960784_0.03137254901960784/linf/1b5250ff-0c88-49fe-bcea-8c571e10c085


Val Epoch:0 | Loss 0.2127 | NatPrec1 93.968 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 10/10 [00:02<00:00,  4.45it/s]
Val Epoch:0 | Loss 75.8394 | AdvPrec1 0.000 | AdvPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 10/10 [00:39<00:00,  3.92s/it]


In [ ]:
# !tensorboard --logdir=/content/drive/MyDrive/Adversarial\ Attacks/robust_transfer_learning/resnet_18l_inf_eval_0.03137254901960784_0.03137254901960784/linf/20696687-fdf7-43f1-82e8-0fd0151b0d58 serve

####ϵ-test: 16/255


In [ ]:
eval_args.__setattr__('constraint', 'inf')
eval_args.__setattr__('eps', 16/255)
print(f'evaluation args: {eval_args}')

#evals on robust model
TEST_L_INF_OUT = OUT_DIR + L_INF_EVAL.format(8/255, 16/255) +'linf'
evaluate_robust_model(eval_args, model, TEST_L_INF_OUT)

evaluation args: {
  "adv_eval": 1,
  "eval_only": 1,
  "use_best": true,
  "random_restarts": 0,
  "constraint": "inf",
  "eps": 0.06274509803921569,
  "attack_steps": 100,
  "attack_lr": 0.000784313725490196,
  "out_dir": "/content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l_inf_eval_0.03137254901960784_0.03137254901960784/linf"
}
evaluating robust model with constaint: inf, eps: 16/255, and attack steps: 100
test results dir: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l_inf_eval_0.03137254901960784_0.06274509803921569/linf
Logging in: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l_inf_eval_0.03137254901960784_0.06274509803921569/linf/66f0e34e-66de-481d-b77d-dd0cdd29d382


Val Epoch:0 | Loss 0.2127 | NatPrec1 93.968 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 10/10 [00:02<00:00,  4.54it/s]
Val Epoch:0 | Loss 94.7035 | AdvPrec1 0.000 | AdvPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 10/10 [00:39<00:00,  3.93s/it]


In [ ]:
# !tensorboard --logdir=/content/drive/MyDrive/Adversarial\ Attacks/robust_transfer_learning/resnet_18l_inf_eval_0.03137254901960784_0.06274509803921569/linf/30aa6236-19a0-4830-b9bb-fba0d691d7c1 serve

##Evaluating robust model with constrain: unconstrained, ϵ-test: unconstrained, and attack_steps: [20, 100]


In [ ]:
# from pyngrok import ngrok

# # Set the ngrok authtoken (only needed for free users)
# NGROK_AUTH_TOKEN = "23Qif7HL0weneUkOUXkCzY3ZRQG_3gPVLhxnC1kRDVRqWezY7"

# ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# # Open a tunnel to the TensorBoard port
# public_url = ngrok.connect(addr="6006", proto="http")

# public_url

In [ ]:
eval_args.__setattr__('constraint', 'unconstrained')
eval_args.__setattr__('eps', 0.1) #eps-values not used
print(f'evaluation args: {eval_args}')

#evals on robust model
TEST_UNCONSTRAINED_OUT = OUT_DIR + UNCONSTRAINED_EVAL + 'unconstrainedyeaa'
evaluate_robust_model(eval_args, model, TEST_UNCONSTRAINED_OUT)

evaluation args: {
  "adv_eval": 1,
  "eval_only": 1,
  "use_best": true,
  "random_restarts": 0,
  "constraint": "unconstrained",
  "eps": 0.1,
  "attack_steps": 100,
  "attack_lr": 0.001568627450980392,
  "out_dir": "/content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18l_inf_eval_0.03137254901960784_0.06274509803921569/linf"
}
evaluating robust model with constaint: unconstrained, eps: 1/10, and attack steps: 100
test results dir: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18unconstrained_eval/unconstrainedyeaa
Logging in: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18unconstrained_eval/unconstrainedyeaa/2ca0c6ed-76c7-41b3-ab72-2d303853c533


Val Epoch:0 | Loss 0.2127 | NatPrec1 93.968 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 10/10 [00:02<00:00,  4.36it/s]
Val Epoch:0 | Loss 6.2360 | AdvPrec1 73.333 | AdvPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 10/10 [00:38<00:00,  3.86s/it]


In [ ]:
# !tensorboard --logdir=/content/drive/MyDrive/Adversarial\ Attacks/robust_transfer_learning/resnet_18unconstrained_eval/unconstrainedyeaa/1458520a-efec-4c73-96f8-0ce62cbea338 serve

##Evaluating robust model with constrain: random smooth, ϵ-test: random smooth and attack_steps: [20, 100]


In [ ]:
# from pyngrok import ngrok

# # Set the ngrok authtoken (only needed for free users)
# NGROK_AUTH_TOKEN = "23Qif7HL0weneUkOUXkCzY3ZRQG_3gPVLhxnC1kRDVRqWezY7"

# ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# # Open a tunnel to the TensorBoard port
# public_url = ngrok.connect(addr="6006", proto="http")

# public_url

In [ ]:
eval_args.__setattr__('constraint', 'random_smooth')
eval_args.__setattr__('eps', 0.1) #eps value not used
print(f'evaluation args: {eval_args}')

#evals on robust model
TEST_RANDOM_OUT = OUT_DIR + RANDOM_EVAL + 'randomsmoothyeaaa'
evaluate_robust_model(eval_args, model, TEST_RANDOM_OUT)

evaluation args: {
  "adv_eval": 1,
  "eval_only": 1,
  "use_best": true,
  "random_restarts": 0,
  "constraint": "random_smooth",
  "eps": 0.1,
  "attack_steps": 100,
  "attack_lr": 0.0025,
  "out_dir": "/content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18unconstrained_eval/unconstrainedyeaa"
}
evaluating robust model with constaint: random_smooth, eps: 1/10, and attack steps: 100
test results dir: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18random_eval/randomsmoothyeaaa
Logging in: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18random_eval/randomsmoothyeaaa/a2e4d314-2beb-4b24-8805-0c8b2b896fd7


Val Epoch:0 | Loss 0.2127 | NatPrec1 93.968 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 10/10 [00:02<00:00,  4.52it/s]
Val Epoch:0 | Loss 0.4527 | AdvPrec1 86.667 | AdvPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 10/10 [00:19<00:00,  1.93s/it]


In [ ]:
# !tensorboard --logdir=/content/drive/MyDrive/Adversarial\ Attacks/robust_transfer_learning/resnet_18random_eval/randomsmoothyeaaa/90e42c70-baa0-40dc-8030-657abea311ac serve

##Evaluating robust model with constrain: fourier, ϵ-test: fourier and attack_steps: [20, 100]


In [ ]:
eval_args.__setattr__('constraint', 'fourier')
eval_args.__setattr__('eps', 0.1)
print(f'evaluation args: {eval_args}')

#evals on robust model
TEST_FOURIER_OUT = OUT_DIR + FOURIER_EVAL
evaluate_robust_model(eval_args, model, TEST_FOURIER_OUT)

evaluation args: {
  "adv_eval": 1,
  "eval_only": 1,
  "use_best": true,
  "random_restarts": 0,
  "constraint": "fourier",
  "eps": 0.1,
  "attack_steps": 100,
  "attack_lr": 0.0025,
  "out_dir": "/content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18random_eval/randomsmoothyeaaa"
}
evaluating robust model with constaint: fourier, eps: 1/10, and attack steps: 100
test results dir: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18fourier_eval/
Logging in: /content/drive/MyDrive/Adversarial Attacks/robust_transfer_learning/resnet_18fourier_eval/774c866a-5921-442f-a943-5f8f4813f6e3


Val Epoch:0 | Loss 0.2127 | NatPrec1 93.968 | NatPrec5 100.000 | Reg term: 0.0 ||: 100%|██████████| 10/10 [00:02<00:00,  3.55it/s]
  0%|          | 0/10 [00:00<?, ?it/s]


AttributeError: ignored